In [27]:
import requests
import os
import time
import pandas
import json

from requests.auth import HTTPBasicAuth
from datetime import datetime
from pandas import DataFrame

test_collection_type = {"1" : "test_run" ,  "2" : "test_plan"}
ODM_testrun_type = {"T" : True, "F": False}
datetime_object = datetime.now()
unix_timestamp = datetime_object.timestamp()
ODM_Testrun_Only = False
days = 30
while True:
    try:
        collection_type = input("Enter collection type: '1 (Test Run) or 2 (Test Plan)':")
        if(collection_type in test_collection_type.keys()):
            break;
        else:
            raise ValueError
    except ValueError:
        print("Wrong Format")

while True:
    try:
        odm_true_or_false= input("Toggle ODM Runs only (T/F):")
        if(odm_true_or_false in ODM_testrun_type.keys()):
            ODM_Testrun_Only = ODM_testrun_type[odm_true_or_false]
            break;
        else:
            raise ValueError
    except ValueError:
        print("Wrong Format") 
    
while True:
    try:
        days = int(input("Enter # of Days Created Before Now: (Ex: 30)"))
        if(days < 0):
            raise ValueError
        break
    except ValueError:
        print("Wrong Format")

test_run_string = [x for x in input("Enter TestRun Filter Strings: ").split()]
testers_id_array = [105, 37, 429, 392, 51, 314, 218, 283, 105]
frames = []
url = ""
testruns = {}
testplans = {}
df = {}
basic = HTTPBasicAuth('jorden.tang@vizio.com', 'Y9MchY52ENet0R/fJIcI-1yBF86Fy8Uhn58dCNiax')

match(test_collection_type[collection_type]): 
    case "test_run":
        url = "https://vizio.testrail.com/index.php?/api/v2/get_runs/44"
        params = {"created_after" : int(unix_timestamp) - days * 24 * 60 * 60}
        res = requests.get(url = url, params=params, auth = basic)
        res_dict = res.json()
        testruns = res_dict["runs"]

        df = DataFrame(testruns)
        df = df.sort_values(by='created_by', ascending=False)
        df = df.filter(items = ['name', 'untested_count', 'passed_count', 'failed_count', 'url', 'created_by'])

        ## replace .com with .io to make url usable
        df['url'] = df['url'].str.replace('.com', '.io')
        condition_contains_test_run_string = df['name'].apply(lambda x: 1 if any(s in x for s in test_run_string ) else 0) 
        condition_tester_id = df['created_by'].isin(testers_id_array) if ODM_Testrun_Only else True
        condition_started_run = df['passed_count'] > 0
        df = df.loc[condition_tester_id & condition_contains_test_run_string & condition_started_run & condition_started_run]

    case "test_plan":
        url = "https://vizio.testrail.com/index.php?/api/v2/get_plans/44"
        params = {"created_after": int(unix_timestamp) - days * 24 * 60 * 60}
        res = requests.get(url = url, params=params, auth = basic)
        res_dict = res.json()
        testplans = res_dict["plans"]

        df = DataFrame(testplans)
        df = df.sort_values(by='created_by', ascending=False)
        df = df.filter(items = ['name', 'untested_count', 'passed_count', 'failed_count', 'url', 'created_by'])
        df['url'] = df['url'].str.replace('.com', '.io')
 
        condition_contains_test_run_string = df['name'].apply(lambda x: 1 if any(s in x for s in test_run_string ) else 0) 
        condition_started_run = df['passed_count'] >= 0
        condition_tester_id = df['created_by'].isin(testers_id_array) if ODM_Testrun_Only else True
        df = df.loc[condition_contains_test_run_string & condition_started_run & condition_tester_id]        

    case _:
        print ("Shouldn't print this")
frames.append(df)
final_result = pandas.concat(frames)
os.makedirs('./results', exist_ok=True) 
current_time = datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
final_result.to_csv('./results/testrail_search_[{search_fields}]_{now}.csv'.format(now = current_time, search_fields = test_run_string))  
